# 模型推理 - 使用QLoRA微调后的ChatGLM3-6B

In [1]:
import os
os.environ['HF_HOME'] = '/root/autodl-tmp/cache/'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'



/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp26': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_ues_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto'
                                      )

/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 7/7 [00:05<00:00,  1.21it/s]
/root/miniconda3/envs/chatglm3demo/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
base_model.requires_grad_(False) # 冻结模型参数（不参与训练）
base_model.eval() # 设为评估模式，dropout关闭

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          )

Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


# 使用原始ChatGLM3-6B模型

In [5]:
input_text = '解释一下乾卦是什么？'

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，它代表了天、代表了刚强、代表了行动和进取。乾卦是由两个阴爻夹一个阳爻构成，象征着天宇的广漠和阳性 principle 的强大。

乾卦的意义非常丰富，它不仅代表了阳刚之气，也代表了天的广大和无限。在易经中，乾卦常常被用来象征父亲、领导、权威和力量。同时，乾卦也代表着一种积极主动的精神，它鼓励人们在天行健的过程中，不断努力，积极进取。

乾卦还有一些特殊的象征意义。例如，乾卦的六爻可以组成一个完整的阳刚之身，代表着一个人的完整和强大。乾卦的九宫也可以代表一个人的九个不同的领域或功能，例如，乾卦的乾宫代表一个人的事业和领导能力，坤卦的坤宫则代表一个人的家庭和情感生活。

总之，乾卦是易经中非常重要的一个八卦，它代表了一种刚强、积极、行动和进取的精神，同时也象征着天的广大和无限。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query='周易中的讼卦是什么？', history=history)
print(response)

《周易》中的讼卦（又称法卦）是八卦之一，它代表了诉讼、争端、法律和规则。讼卦是由两个阴爻夹一个阳爻构成，象征着阴阳相争，纷争不断。

讼卦的意义主要体现在以下几个方面：

1. 诉讼和争端：讼卦提醒人们在处理争端和诉讼时要保持冷静、公平和依法办事。它告诉人们，在争端中要遵循道义和公平的原则，以达到和谐共处的目的。

2. 法律和规则：讼卦也象征着法律和规则的权威。它告诉我们，在解决问题和争端时，要遵循既定的法律法规和游戏规则，不能随意篡改。

3. 阴阳平衡：讼卦中的阴阳代表了两种相对立的势力。它告诫我们在处理争端和诉讼时，要寻求阴阳平衡，努力实现和谐共处。

4. 成长和进退：讼卦还可以看作是成长和进退的象征。它告诉我们，在面对困难和挑战时，要勇敢面对，积极寻求解决办法，不断成长和进步。

总之，讼卦在周易中具有重要地位，它提醒人们在处理争端和诉讼时要保持冷静、公平和依法办事，同时在解决问题时要努力实现阴阳平衡，寻求和谐共处。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = '20250623_155120'

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automode-dataset(fixed))-{timestamp}"


In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)
    
    inputs = tokenizer(query, return_tensors='pt').to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n微调后{training_tag}：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。

微调后ChatGLM3-6B(Epoch=3, automode-dataset(fixed))-20250623_155120：
[gMASK] sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。

微调后ChatGLM3-6B(Epoch=3, automode-dataset(fixed))-20250623_155120：
[gMASK] sop 周易中的讼卦是什么？ 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [13]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = '20250623_153558'
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [14]:
base_response, ft_response = compare_chatglm_results("解释一下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释一下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，象征刚健强劲，代表着阳刚之气。它的卦象为乾，由六个阳爻组成，代表着刚健强劲，发展壮丽。乾卦的哲学内涵是：刚健者，敬德广布，大行天下。

在周易中，乾卦被解释为天，具有刚健强劲的品质，代表著阳刚之气。天是刚强劲健的，万物由此而成立，故称为“天行健”。天行健，万物得时，故顺其自然，并不妄为。

乾卦的取象为天，象征刚健强劲，代表着阳刚之气。它代表著力量的储备，如同储备的精力、能力等，等待时机， spoken

微调后ChatGLM3-6B(Epoch=3, automade-dataset)：
[gMASK] sop 解释一下乾卦是什么？ 乾卦"是周易中的开山之作，由六个阳爻（代表天）和五个阴爻（代表地）组成，象征天公无私，刚毅果敢，统一领导下的积极景象。在卜筮中，乾卦预示着刚毅果敢，有利于统一领导下的积极景象，同时也预示着前进将面临困难。


In [15]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易卦象中的一卦，由地水两种元素相结合而成。在周易中，师卦代表军队、大众、大众 water（水）师，预示着战争顺利，众志成城。在卜问中，预示着解答谜题，获取资讯，但也有可能因为迷途而陷入困境。

师卦的核心哲学是：天行健，君子以自强不息。君子观此卦象，应该保持积极进取的精神，勇敢地面对各种困难，以自身的努力和坚持来化解困境。

另外，这个卦象也预示着即将遇到的困难，需要大家齐心协力，共同克服。在事业、经商、求名、婚恋等方面会有很好的表现，但也要谨慎，避免陷入不必要的麻烦。

微调后ChatGLM3-6B(Epoch=3, automade-dataset)：
[gMASK] sop 地水师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，地水师卦的哲学含义为：“地水相承，志在养民。圣人之德，在养民成德。”意味着以德治国，强调尊重和关爱人民群众，根据情况灵活应对，从而达到顺利和谐的目的。


在传统解卦中，地水师卦被认为具有养民、顺天、平乱的功效，对个人和社会具有积极的意义。


In [16]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。

讼卦的经商指引是：遇到困难，靠诚实为本，坚持正派经营方式。在商业谈判中要坚持原则，避免因为利益冲突而陷入诉讼。

讼卦对个人的启示是：面对争端和冲突时，要冷静分析，退让一步，求得和解。坚持正义和正派行为，避免陷入诉讼纠纷。

讼卦强调的是退让和求和，而不是固执己见，这是其核心哲学。在遇到困难和挫折时，要善于调整策略，寻求更好的解决办法。只有这样，才能在复杂多变的局势中保持稳定和安全。

微调后ChatGLM3-6B(Epoch=3, automade-dataset)：
[gMASK] sop 周易中的讼卦是什么样子 的卦象是上卦为乾（天），下卦为坤（地），两者相背而行，天地的相背，代表天刚健，地柔顺，刚柔相济，在天长养，在地成泉。讼卦中，天刚健，地柔顺，刚柔相济，预示着天象刚健，地景柔顺，刚柔相济，预示着吉利。

讼卦的核心哲学是：天刚健，地柔顺，刚柔相济，在天长养，在地成泉。天象刚健，地景柔顺，刚柔相济，预示着吉利。

讼卦的吉利在于天刚健，地柔顺，刚柔相济，预示着吉利。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
timestamp = '20250623_172003'
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"


In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，也是八宫图上的第一个卦。乾卦是由两个阳爻夹一个阴爻构成，象征着天、云和雨，以及君王、领导、权力等。乾卦的卦辞是：“乾，元，亨，利，贞。”这意味着乾卦是至高无上的，能够通行无阻，带有利益，是正确的。

乾卦的卦象是由两个阳爻夹一个阴爻构成，象征着天，其中阳爻代表天，阴爻代表地。乾卦的卦象是由三个阴爻夹一个阳爻构成，这意味着天地的配合，也象征着君王和领导。乾卦还象征着权力、权威和统治，这些特性使得乾卦成为一种非常有力量和权威的卦。

乾卦的卦辞中，“元”表示至高无上的地位，“亨”表示通行无阻， “利”表示有利可图，“贞”表示正确。这些特性使得乾卦具有非常高的地位和权威，成为至高无上的卦。

微调后ChatGLM3-6B(Epoch=50, handmade-dataset)：
[gMASK] sop 解释下乾卦是什么？ 乾卦是《易经》中的一种卦象，它是由六个阳爻（表示刚强、积极、正义等品质）和一条阴爻（表示柔弱、消极、不正义等品质）组成的。阴爻位于卦象的下方，阳爻位于卦象的上方。乾卦的卦象是由两个阴爻夹一个阳爻构成，象征着天（乾为天，阴为地，阴阳相济），也象征着君子，以及刚强、积极、正义的品质。乾卦的卦辞：“乾，元，亨，利，贞。”表示乾卦象征天，具有无穷的力量，能够通達良好的结果，吉凶祸福取决于阴爻和阳爻的位置。乾卦的卦辞：“乾。元，亨，利，贞。象曰：天行健，君子以自强不息。”，表明乾卦象征天，君子应该效法天的精神，自强不息。


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦是卦象，由下卦坎（水）上卦坤（地），坤为地（顺），坎为水（险）。师卦象征众军captain，君子以测征。

师卦的卦辞：“师，众也。有众直止，不见其增，有所不为。

象曰：地中有水，师。君子以容众。。”

师卦的主卦为坤卦（地），副卦为坎卦（水）。

坤卦（地）卦辞：“坤，元，亨，利，贞。

象曰：坤为地，顺为正，承天行义。。”

坎卦（水）卦辞：“坎，云，雨，利，derivative。

象曰：坎卦下卦坎上卦坤，具象地(象征)川流，险而益坚，先倾覆后益坚。

附：

“师”有军队作战的含義，此卦为坤地（顺），有众军直止之象，可见君子有包容容纳众人的德行。

微调后ChatGLM3-6B(Epoch=50, handmade-dataset)：
[gMASK] sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦是《易经》中第六卦，卦辞：“讼，有孚，窒惕，中吉，终凶。利见大人，不利涉大川。”

讼卦象征诉讼，但在《易经》中，卦辞中的“有孚”，说明诉讼是为了追求真理和正义。 timidity，警惕，中吉，终凶： timidity指谨慎，警惕，能够带来吉祥，但过于谨慎则凶。


卦象：上卦为乾，下卦为坤。乾为阳，坤为阴。乾卦象征天，坤卦象征地。乾卦的阳刚之象与坤卦的阴柔之象结合在一起，象征天地相辅，刚柔并济。

应用：讼卦讲求中庸之道，过于刚健或柔顺都会带来危险。在现实生活中，应根据具体情况，恰当运用刚柔之力，才能获得良好的结果。

微调后ChatGLM3-6B(Epoch=50, handmade-dataset)：
[gMASK] sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离君子，君子需慎重谋事。台湾国学大儒傅佩荣解：时运遇阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
